<a href="https://colab.research.google.com/github/mofuoku417/anomaly-detection/blob/main/Detecting_Anomalies_In_Natural_Gas_Prices_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Problem statement**

This project aims to analyze crude oil data from 01/04/200 - 6/17/22 using anomaly detection techniques to uncover hidden patterns and outliers in the data. We will use the Pytorch library in Python to construct and train a model to detect anomalous data points within the dataset. Finally, we will analyze and interpret our results to draw meaningful conclusions about the Crude oil market.

####Loading the required libraries

In [3]:
from pandas import read_csv
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

####Loading the dataset

In [4]:
#Load dataset
path = 'https://raw.githubusercontent.com/mofuoku417/anomaly-detection/main/Data/HHM.csv'
dataframe = read_csv(path, parse_dates=True,)

###Data Preprocessing

In [5]:
print(f"Downloaded {len(dataframe)} rows of data.")
dataframe

Downloaded 318 rows of data.


,Date,MHHS
0,Jan-97,3.45
1,Feb-97,2.15
2,Mar-97,1.89
3,Apr-97,2.03
4,May-97,2.25
...,...,...
313,Feb-23,2.38
314,Mar-23,2.31
315,Apr-23,2.16
316,May-23,2.15


In [6]:
#Checking for missing values
dataframe.isnull().sum()

Date    0
MHHS    0
dtype: int64

There are no missing values

In [7]:
#Checking the data types
dataframe.dtypes

Date     object
MHHS    float64
dtype: object

In [10]:
dataframe.head(10)

,Date,MHHS
0,Jan-97,3.45
1,Feb-97,2.15
2,Mar-97,1.89
3,Apr-97,2.03
4,May-97,2.25
5,Jun-97,2.20
6,Jul-97,2.19
7,Aug-97,2.49
8,Sep-97,2.88
9,Oct-97,3.07


####Changing the ‘Date Starting’ column to a date datatype:

In [9]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

ParserError: ignored

In [ ]:
# Work in progress